In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_Anaheim

In [2]:
numNodes = Compute_Jacobian_Anaheim.numNodes;
numLinks = Compute_Jacobian_Anaheim.numLinks;
numODpairs = Compute_Jacobian_Anaheim.numODpairs;
numZones = Compute_Jacobian_Anaheim.numZones;
od_pairs = Compute_Jacobian_Anaheim.od_pairs;
link_list_js = Compute_Jacobian_Anaheim.link_list_js;
link_length_list = Compute_Jacobian_Anaheim.link_length_list;

In [3]:
include("../../Julia_files/initia_data.jl");
include("../../Julia_files/load_network_uni_class.jl");
include("prepare_data_Anaheim.jl");
include("extract_data_Anaheim.jl");
include("../../Julia_files/tap_MSA.jl");
include("../../Julia_files/inverseVI.jl");
include("../../Julia_files/demands_adjustment_gradi.jl");

In [4]:
fcoeffs = [1, 0, 0, 0, .15];

# get observed flow vector (corresponding to ground truth demands and ground truth costs)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [5]:
# get arcs data corresponding to ground truth demands and flows
arcsDict[0] = observFlow("./benchmark_data/Anaheim_net.txt", tapFlowDicDict[0])

Dict{Any,Any} with 914 entries:
  (71,255)  => Arc(71,255,1800.0,0.5,2256.86)
  (323,324) => Arc(323,324,5400.0,0.5,1101.68)
  (354,353) => Arc(354,353,5400.0,1.0,8.1826)
  (60,230)  => Arc(60,230,9000.0,0.333333,2184.31)
  (223,346) => Arc(223,346,1800.0,0.5,1407.46)
  (147,146) => Arc(147,146,7200.0,0.763321,3563.88)
  (337,33)  => Arc(337,33,12600.0,0.149068,6581.72)
  (366,365) => Arc(366,365,5400.0,0.5,1008.7)
  (390,375) => Arc(390,375,5400.0,2.0,0.0)
  (315,316) => Arc(315,316,5400.0,2.57992,0.0)
  (198,197) => Arc(198,197,7200.0,0.152623,6072.44)
  (333,334) => Arc(333,334,5400.0,0.420076,1501.87)
  (367,368) => Arc(367,368,5400.0,0.720076,132.977)
  (244,339) => Arc(244,339,1800.0,0.5,1770.03)
  (274,41)  => Arc(274,41,5400.0,1.0,0.0)
  (225,330) => Arc(225,330,1800.0,0.5,997.887)
  (49,369)  => Arc(49,369,5400.0,1.0,0.0)
  (274,275) => Arc(274,275,5400.0,2.0,501.6)
  (15,254)  => Arc(15,254,5400.0,1.0,407.1)
  (226,225) => Arc(226,225,9000.0,0.937836,5169.63)
  (218,392) => A

In [6]:
deg_list = [3, 4, 5, 6]
c_list = [0.5, 1.0, 1.5]
lam_list = [0.01, 0.1, 1.0, 10.0, 100.0]

for deg in deg_list
    for c in c_list
        for lam in lam_list
            coeffs_dict[(deg, c, lam, 1)], objInvVIDict[1] = train(lam, deg, c, demandsDict[0], arcsDict[0]);
        end
    end
end

In [7]:
outfile = open("./results_Anaheim/coeffs_dict_Anaheim.json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)